In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
lemm = WordNetLemmatizer()

lines=100000

In [2]:

def create_lexicon():
    lexicon=[]
    with open('pos.txt','r') as f:
        contents = f.readlines()
        #all_words=word_tokenize(contents)
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon+=list(all_words)

                
                    
            
    with open('neg.txt','r') as f:
        contents = f.readlines()
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon += list(all_words)
    
    lexicon = [lemm.lemmatize(i,pos='n') for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        #print(w_counts[w])
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    return l2

In [3]:
def make_features(sample,lexicon,char):
    import numpy as np
    featureset=[]
    with open(sample,'r')as f:
        contents=f.readlines()
        
        for l in contents[:lines]:
            sam_words=word_tokenize(l)
            sam_words=[lemm.lemmatize(i) for i in sam_words]
            features=np.zeros(len(lexicon))
            for word in sam_words:
                if word.lower() in lexicon:
                    index_value=lexicon.index(word.lower())
                    features[index_value]+=1
            features=list(features)
            featureset.append([features,char])
    return featureset

In [45]:
import pandas as pd
data=pd.read_csv(r"C:\Users\PIYUSH\Desktop\news.csv")

In [46]:
len(data["headline"].unique())

2873

In [50]:
news=data["text"]
headline=data["headline"]
news[0]

"NEW YORK (AP) - A prime rate reduction by New York's First National City Bank helped the stock market close out the busiest week in its history with a modest advance Friday.</br></br>The Dow Jones average of 30 industrial stocks gained 5.12 to 929.63, finishing the week withanetrise of 18.50 points.</br></br>Big Board volume, at 25.94 million shares, fell a good deal short of matching Thursday's all-time high of 38.45 million.</br></br>But the day�۪s figure was more than enough to push the total for the week to a record 159.69 million shares. The previous peak had been 145.68 million in the last week of January 1975.</br></br>The Dow has advanced in all but two of the first 11 sessions of the new year. As of Friday�۪s close it had gained 77.22 points since New Year�۪s."

In [42]:
def data_prepare(headline):
    lexicon=[]
    for i in headline:
        all_words=word_tokenize(i)
        lexicon+=list(all_words)
    head_lexicon=[]
    stop = set(stopwords.words('english'))
    head_lexicon+=[i for i in lexicon if i not in stop]
    head_lexicon = [lemm.lemmatize(i,pos='n') for i in head_lexicon]
    head_words=[[word.lower() for word in head_lexicon if word.isalpha()]]
    return head_words

In [44]:
head_words=data_prepare(headline)
head_lexicons=(head_words)

In [40]:
lexicons=[]
for i in news:
    all_words=word_tokenize(i)
    lexicons+=list(all_words)
text_lexicon=[]
stop = set(stopwords.words('english'))
text_lexicon+=[i for i in lexicons if i not in stop]
text_lexicon = [lemm.lemmatize(i,pos='n') for i in text_lexicon]
text_words=[[word.lower() for word in text_lexicon if word.isalpha()]]

    

In [41]:
text_words

[['new',
  'york',
  'ap',
  'a',
  'prime',
  'rate',
  'reduction',
  'new',
  'york',
  'first',
  'national',
  'city',
  'bank',
  'helped',
  'stock',
  'market',
  'close',
  'busiest',
  'week',
  'history',
  'modest',
  'advance',
  'the',
  'dow',
  'jones',
  'average',
  'industrial',
  'stock',
  'gained',
  'finishing',
  'week',
  'withanetrise',
  'big',
  'board',
  'volume',
  'million',
  'share',
  'fell',
  'good',
  'deal',
  'short',
  'matching',
  'thursday',
  'high',
  'but',
  'figure',
  'enough',
  'push',
  'total',
  'week',
  'record',
  'million',
  'share',
  'the',
  'previous',
  'peak',
  'million',
  'last',
  'week',
  'january',
  'the',
  'dow',
  'advanced',
  'two',
  'first',
  'session',
  'new',
  'year',
  'as',
  'close',
  'gained',
  'point',
  'since',
  'new',
  'mayor',
  'anthony',
  'williams',
  'petitioned',
  'white',
  'house',
  'yesterday',
  'million',
  'federal',
  'relief',
  'five',
  'time',
  'official',
  'originall

In [38]:
from gensim.models import Word2Vec

In [39]:
s=[["sale","maximum"]]
model = Word2Vec(s, min_count=1,)
print(model)

Word2Vec(vocab=2, size=100, alpha=0.025)


In [37]:
words = list(model.wv.vocab)
print(words)

['sale', 'maximum']


In [33]:
model['maximum']

c:\users\piyush\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.00971811, -0.0193586 , -0.00715668, -0.00477168,  0.01329297,
       -0.00702609,  0.00835602, -0.00875634,  0.00922986,  0.01712107,
       -0.00285383, -0.00905806,  0.01514944,  0.00746247,  0.00160464,
       -0.0066373 ,  0.01667181,  0.00379292,  0.01043171,  0.01539471,
        0.00989748, -0.01223098,  0.00504088, -0.00722353, -0.00121213],
      dtype=float32)